Reference: https://github.com/karpathy/ng-video-lecture

In [1]:
import torch

In [2]:
# hyperparameters

batch_size = 32 # how many independent sequences will we process in parallel
block_size = 8 # what is the maximum context length for prediction
eval_interval = 1000
max_iters = 50000
n_embd = 256
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'


In [3]:
# Read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("Length of dataset in characters: ", len(text))


Length of dataset in characters:  1115394


In [4]:
# Let's look at the first 1000 characters
print(text[:1000])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
# Here are all the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('Vocabulary size:', vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Vocabulary size: 65


In [6]:
# Create a mapping from character to integer
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[ch] for ch in s] # encoder: takes a string and returns a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integers and returns a string

print(encode("hii there"))
print(decode(encode("hii there")))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
"""
Other mapping methods:
    1. Google: sentencepiece
               https://github.com/google/sentencepiece

    2. OpenAI: tiktoken
               https://github.com/openai/tiktoken

               import tiktoken
               enc = tiktoken.get_encoding("gpt2")
               enc.n_vocab
               enc.encode("hii there")
               enc.decode([71, 4178, 612])

"""


'\nOther mapping methods:\n    1. Google: sentencepiece\n               https://github.com/google/sentencepiece\n\n    2. OpenAI: tiktoken\n               https://github.com/openai/tiktoken\n\n               import tiktoken\n               enc = tiktoken.get_encoding("gpt2")\n               enc.n_vocab\n               enc.encode("hii there")\n               enc.decode([71, 4178, 612])\n\n'

In [8]:
# Let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
data = data.to(device)
print(data.shape, data.dtype)
print(data[:100]) # the 100 chatacters we looked at


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% of the data will be used for training, rest for validation
train_data, valid_data = data[:n], data[n:]


In [10]:
valid_data.is_cuda

False

In [11]:
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

"""
Let the transformer predict next character by input 1 character.
And predict next character by input 2 characters.
And predict next character by input 3 characters.
And so on.
"""


when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


'\nLet the transformer predict next character by input 1 character.\nAnd predict next character by input 2 characters.\nAnd predict next character by input 3 characters.\nAnd so on.\n'

In [13]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of inputs x and targets y
    if split == 'train':
        data = train_data
    else:
        data = valid_data

    ix = torch.randint(data.size(0) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix]) # (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # shift 1 position to right

    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)

print('targets:')
print(yb.shape)
print(yb)

print('------')

for b in range(batch_size): # batch_dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

inputs:
torch.Size([32, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
      

In [14]:
yb.is_cuda

False

In [15]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # self.position_embedding_table = nn.Embedding(block_size, n_embd) # include the character position
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        # pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        # x = tok_emb + pos_emb # (B,T,C)
        # logits = self.lm_head(x) # (B,T,vocab_size)
        logits = self.lm_head(tok_emb)

        if targets is None:
            loss = None
        else:
            # because the second dimension of cross_entorpy represents the channels
            # we need to reshape the logits and targets
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):

        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # become (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            next_tokens = torch.multinomial(probs, num_samples=1) # (B,1)
            # append smapled index to the running sequence
            idx = torch.cat([idx, next_tokens], dim=-1) # (B,T+1)

        return idx
    
model = BigramLanguageModel()
model = model.to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)
start_idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(model.generate(start_idx, max_new_tokens=100)[0].tolist()))
# beacuse the model is not trained yet, the output is random
# also, we only focus on the last time step, did not consider the history 


torch.Size([256, 65])
tensor(4.3727, grad_fn=<NllLossBackward0>)

HPlMBWro;w;UATy!dSD3UHrRygABctWrwezjJ-JyYkez kvlVuD;&YjT bl.K?;HoXSHWtwSoGsXXa-.?DuMm3ylY?DRijBqR;VR


In [16]:
model.generate(start_idx, max_new_tokens=100)[0].tolist()

[0,
 13,
 45,
 2,
 13,
 39,
 60,
 6,
 19,
 57,
 34,
 40,
 20,
 42,
 49,
 27,
 38,
 36,
 20,
 47,
 14,
 63,
 3,
 32,
 3,
 44,
 58,
 49,
 32,
 17,
 42,
 12,
 20,
 28,
 9,
 64,
 63,
 31,
 13,
 43,
 58,
 57,
 27,
 15,
 46,
 20,
 28,
 7,
 38,
 64,
 42,
 57,
 51,
 35,
 48,
 47,
 5,
 57,
 0,
 40,
 64,
 13,
 3,
 48,
 32,
 30,
 0,
 42,
 63,
 16,
 33,
 60,
 33,
 6,
 20,
 38,
 54,
 52,
 64,
 58,
 7,
 30,
 59,
 13,
 34,
 39,
 55,
 6,
 57,
 52,
 63,
 50,
 42,
 45,
 8,
 27,
 26,
 57,
 36,
 19,
 36]

In [17]:
# create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)


In [18]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval() # change to eval mode
    for split in ['train', 'valid']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            x, y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()

    return out


In [19]:
for steps in range(max_iters):

    # every once in a while evaluate the loss on train and val loss
    if steps % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {steps}, train loss: {losses['train']:.4f}, val loss: {losses['valid']:.4f}")
        
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.3391, val loss: 4.3362
step: 1000, train loss: 2.4851, val loss: 2.5137
step: 2000, train loss: 2.4850, val loss: 2.5183
step: 3000, train loss: 2.4790, val loss: 2.5218
step: 4000, train loss: 2.4758, val loss: 2.4987
step: 5000, train loss: 2.4699, val loss: 2.5050
step: 6000, train loss: 2.4763, val loss: 2.5090
step: 7000, train loss: 2.4742, val loss: 2.4936
step: 8000, train loss: 2.4723, val loss: 2.5128
step: 9000, train loss: 2.4728, val loss: 2.5128
step: 10000, train loss: 2.4792, val loss: 2.4988
step: 11000, train loss: 2.4740, val loss: 2.5024
step: 12000, train loss: 2.4723, val loss: 2.4989
step: 13000, train loss: 2.4722, val loss: 2.4999
step: 14000, train loss: 2.4728, val loss: 2.5017
step: 15000, train loss: 2.4696, val loss: 2.5052
step: 16000, train loss: 2.4742, val loss: 2.4976
step: 17000, train loss: 2.4699, val loss: 2.4942
step: 18000, train loss: 2.4701, val loss: 2.5049
step: 19000, train loss: 2.4690, val loss: 2.4988
step: 20000, 

In [20]:
print(decode(model.generate(start_idx, max_new_tokens=400)[0].tolist()))
# characters are not talk to each other


Alorn. I ann
A f whenes inghesche FR us.'d louthis cet this;
AUCE:
ARY:
Se,---weaxt thorye y lur cr thy.
brst ts wim nd.
Rownsam t, bend yodeanfutheloneswe wndy. th cco we haper ipl ilioovith r.' f
TO,
Beeancit afow wor,
Prd
Two wate, mar obers atold d y, spou hal d:
INRK:
Angred t o y'd, afry wa S:
Byafe'l p sharindistucencorglin!
Rothoue ar, al by owhar;
Tofrstithaughemouthos hefof pllyomanch th
